In [2]:
!pip list | grep torch
!pip list | grep transformers
!pip list | grep fugashi
!pip list | grep ipadic

torch                        2.0.0
transformers                 4.28.1
fugashi                      1.2.1
ipadic                       1.0.0


In [13]:
import os
from glob import glob
import pandas as pd
import linecache

#from transformers.modeling_bert import BertModel
from transformers import BertJapaneseTokenizer

tokenizer = BertJapaneseTokenizer.from_pretrained('cl-tohoku/bert-base-japanese-whole-word-masking')

# カテゴリを配列で取得
categories = [name for name in os.listdir("text") if os.path.isdir("text/" + name)]
print(categories)
# ['movie-enter', 'it-life-hack', 'kaden-channel', 'topic-news', 'livedoor-homme', 'peachy', 'sports-watch', 'dokujo-tsushin', 'smax']

datasets = pd.DataFrame(columns=["title", "category"])
for cat in categories:
    files = glob(f'text/{cat}/*.txt')
    for text_name in files:
        title = linecache.getline(text_name, 3)
        wakati_ids = tokenizer.encode(title, return_tensors='pt')
        #print(*tokenizer.convert_ids_to_tokens(wakati_ids[0].tolist()))
        datasets.loc[len(datasets)] = pd.Series([title, cat], index=datasets.columns)

# データフレームシャッフル
datasets = datasets.sample(frac=1).reset_index(drop=True)

datasets.head()
print(datasets)

datasets.to_pickle('livedoor_data.pickle')

#title  category
#0  兼用アンテナ搭載の「Viewer Dock」が同梱！シャープのドコモ向けハイエンドエンタメ系... smax
#1  女は“愛嬌”、男も“愛嬌”-人事担当者がこっそり教える採用ウラ話 vol.6\n  livedoor-homme
#2  社会貢献×ファッションがカッコイイ、今年の春旋風を巻き起こしたMODE for Charit...  peachy
#3  今でも、後でも読めるニュースがここにある！スマホでもタブレットでも読みやすいITニュース活用...   it-life-hack
#4  被災地の缶詰を途上国に…「正気じゃない。人殺しだ!!」\n topic-news

['it-life-hack', 'dokujo-tsushin', 'livedoor-homme', 'movie-enter', 'peachy', 'topic-news', 'kaden-channel', 'smax', 'sports-watch']
                                                  title        category
0     本日発売開始！au初Android 4.0スマートフォン「HTC J ISW13HT」で本体...            smax
1     【Office Scope vol.03】ITという海原を往く、遊びゴコロたっぷりの「マザー...  livedoor-homme
2                         せっかくの3D恐怖動画なのに、はしゃぎ過ぎの満島ひかり\n     movie-enter
3     イー・モバイルオンラインストアがお得なアウトレットモールをオープン！直販のお得度を早速チェック\n            smax
4                      全米No.1映画『リアル・スティール』の予告＆ポスターが公開\n     movie-enter
...                                                 ...             ...
7371              肩や腕の負担を軽減するExcel／Wordキー操作術【知っ得！虎の巻】\n    it-life-hack
7372                             ″20代の4割が童貞・処女″にネット騒然\n      topic-news
7373           林家三平＆国分佐智子出演『カーズ2』CMのラブラブなメイキング映像を先行公開\n     movie-enter
7374                時代を変えた一冊、『ヘルプ　〜心がつなぐストーリー〜』予告編が解禁\n     movie-enter
7375                 関西エリアが予感させる、「auスマートバリュー」の破壊力と可能性\n    it-life-hack

[7

In [2]:
import numpy as np
import pandas as pd
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext

# データ読み込み
with open('livedoor_data.pickle', 'rb') as r:
    livedoor_data = pickle.load(r)

# データ確認
display(livedoor_data.head())

# 正解ラベル（カテゴリー）をデータセットから取得
categories = list(set(livedoor_data['category']))
print(categories)

# カテゴリーのID辞書を作成
id2cat = dict(zip(list(range(len(categories))), categories))
cat2id = dict(zip(categories, list(range(len(categories)))))
print(id2cat)
print(cat2id)

# DataFrameにカテゴリーID列を追加
livedoor_data['category_id'] = livedoor_data['category'].map(cat2id)

# 念の為シャッフル
livedoor_data = livedoor_data.sample(frac=1).reset_index(drop=True)

# データセットを本文とカテゴリーID列だけにする
livedoor_data = livedoor_data[['title', 'category_id']]
display(livedoor_data.head())

,title,category
0,本日発売開始！au初Android 4.0スマートフォン「HTC J ISW13HT」で本体...,smax
1,【Office Scope vol.03】ITという海原を往く、遊びゴコロたっぷりの「マザー...,livedoor-homme
2,せっかくの3D恐怖動画なのに、はしゃぎ過ぎの満島ひかり\n,movie-enter
3,イー・モバイルオンラインストアがお得なアウトレットモールをオープン！直販のお得度を早速チェック\n,smax
4,全米No.1映画『リアル・スティール』の予告＆ポスターが公開\n,movie-enter


['peachy', 'dokujo-tsushin', 'movie-enter', 'topic-news', 'livedoor-homme', 'kaden-channel', 'sports-watch', 'smax', 'it-life-hack']
{0: 'peachy', 1: 'dokujo-tsushin', 2: 'movie-enter', 3: 'topic-news', 4: 'livedoor-homme', 5: 'kaden-channel', 6: 'sports-watch', 7: 'smax', 8: 'it-life-hack'}
{'peachy': 0, 'dokujo-tsushin': 1, 'movie-enter': 2, 'topic-news': 3, 'livedoor-homme': 4, 'kaden-channel': 5, 'sports-watch': 6, 'smax': 7, 'it-life-hack': 8}


,title,category_id
0,NTTドコモ、利用期間10年以上のユーザーを対象にXiスマートフォン購入で「月々サポート」を...,7
1,女医が教えるオンナの体ウソホント vol.4「イク！って何？」 Presented by ゆ...,1
2,幸せになるコツはデートをリードすること\n,1
3,【Sports Watch】世界で一番かわいい力持ち\n,6
4,NTTドコモ、今夏最注目スマホ「GALAXY SIII SC-06D」に不具合でソフトウェア...,7
